In [1]:
import h5py
import numpy as np
from build_dataset import PoseVideo
import os
import moviepy.editor as mpy
import tensorflow as tf
from keras import layers
from keras import models
from keras import optimizers
import keras.backend as K
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.draw import circle_perimeter, circle





Using TensorFlow backend.


In [4]:
def get_heatmaps_from_video(video_path, model_path):
    
    model =  models.load_model(model_path, custom_objects={'tf':tf, 
                                                             'cv2':cv2,
                                                             'total_frames':5, 
                                                             'num_bodyparts':6, 
                                                             'frame_height':224,
                                                             'frame_width':224,
                                                             'frame_channels':3})

    vid_data = PoseVideo(video_path)
    labelled_frames = []
    frames = []
    for batch in vid_data.predict_data:
#         labelled_frames1.extend(model.predict_on_batch(batch)[0])
#         print(model.predict_on_batch(batch)[1].shape)
        labelled_frames.extend(model.predict_on_batch(batch))
        frames.extend(batch)
        
    return {'heatmaps' : labelled_frames, 'frames' : frames} 

def find_peaks(frame):
    image_max = ndi.maximum_filter(frame, size=15)
    #print('image_max ', image_max)
    coordinates = peak_local_max(np.squeeze(frame), threshold_abs=0.3,  min_distance=15)
    #plt.imshow(np.squeeze(image_max))
    
    return {'coordinates' : coordinates}


def get_pos_from_heatmaps(labelled_frames):
    coordinates = []
    for frame in labelled_frames:
        data = find_peaks(frame)
        coordinates.append(data)
    
    return coordinates
    
def get_flow_from_heatmaps(heatmap_arr):
    pass

def yield_heatmap_batch(heatmap_arr):
    final  = len(heatmap_arr['heatmaps'])
    batchsize = 5
    start = 0
    while(1):
        if start >= final-batchsize:
            return np.array(heatmap_arr['heatmaps'][start:final])
        elif start < final -  batchsize:
            yield np.array(heatmap_arr['heatmaps'][start:start+batchsize])
            start +=  batchsize
            
def predict_heatmaps_from_video(model_path, heatmap_arr):
    
    model =  models.load_model(model_path, custom_objects={'tf':tf, 
                                                             'cv2':cv2,
                                                             'total_frames':5, 
                                                             'num_bodyparts':6, 
                                                             'frame_height':224,
                                                             'frame_width':224,
                                                             'frame_channels':3})
    
    vid_data = yield_heatmap_batch(heatmap_arr)
    central_heatmaps = []
    while(1):
        try:
            data = next(vid_data)
            central_heatmaps.append(model.predict_on_batch(np.expand_dims(data, axis=0)))
        except StopIteration:
            return central_heatmaps
    return central_heatmaps
        

In [5]:
video_path = r"/home/bhandawatlab_duke/deep-pose-predict/181023_1_12_video2.avi"
heatmap_model_path = r"/home/bhandawatlab_duke/Pose-Estimation-Keras/pretrained_models/heatmap_generator_1.h5"
flow_model_path = r"/home/bhandawatlab_duke/Pose-Estimation-Keras/pretrained_models/optical_flow_corrector_1.h5"

heatmap_arr =  get_heatmaps_from_video(video_path, heatmap_model_path)

In [ ]:
print('Total Heatmaps', len(heatmap_arr['heatmaps']))
print('Shape of each heatmap', heatmap_arr['heatmaps'][1].shape)

Total Heatmaps 1935
Shape of each heatmap (224, 224, 6)


In [ ]:
central_heatmaps = predict_heatmaps_from_video(flow_model_path, heatmap_arr)

In [ ]:
print('Total Heatmaps', len(central_heatmaps))
print('Shape of each heatmap', central_heatmaps[1].shape)

In [ ]:
plt.imshow(np.squeeze(central_heatmaps[1][0, ..., 1]))

In [ ]:
plt.imshow(heatmap_arr['heatmaps'][5][:, :, 5])